# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
import time
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
from env import GameEnv
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [2]:
envs = []
for i in range(num_envs):
    envs.append(GameEnv('SpaceInvadersDeterministic-v4'))
#env.render()

/home/sigai/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [3]:
number_lives = envs[0].life
state_size = envs[0].observation_space.shape
action_size = envs[0].action_space.n
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [4]:
agent = Agent(action_size)
torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0
reset_max = 10

### Main Training Loop

In [5]:
vis_env_idx = 0
vis_env = envs[vis_env_idx]
e = 0
frame = 0
max_eval = -np.inf
reset_count = 0

while (frame < 10000000):
    step = 0
    assert(num_envs * env_mem_size == train_frame)
    frame_next_vals = []
    for i in range(num_envs):
        env = envs[i]
        #history = env.history
        #life = env.life
        #state, reward, done, info = [env.state, env.reward, env.done, env.info]
        for j in range(env_mem_size):
            step += 1
            frame += 1
            
            curr_state = env.history[HISTORY_SIZE-1,:,:]
            action, value = agent.get_action(np.float32(env.history[:HISTORY_SIZE,:,:]) / 255.)
            
            next_state, env.reward, env.done, env.info = env.step(action)
            
            if (i == vis_env_idx):
                vis_env._env.render()
            
            frame_next_state = get_frame(next_state)
            env.history[HISTORY_SIZE,:,:] = frame_next_state
            terminal_state = check_live(env.life, env.info['ale.lives'])
            
            env.life = env.info['ale.lives']
            r = env.reward
            
            agent.memory.push(i, deepcopy(curr_state), action, r, terminal_state, value, 0, 0)
            if (j == env_mem_size-1):
                _, frame_next_val = agent.get_action(np.float32(env.history[1:,:,:]) / 255.)
                frame_next_vals.append(frame_next_val)
            env.score += r
            env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
            
            if (env.done):
                if (e % 50 == 0):
                    print('now time : ', datetime.now())
                    rewards.append(np.mean(evaluation_reward))
                    episodes.append(e)
                    pylab.plot(episodes, rewards, 'b')
                    pylab.savefig("./save_graph/spaceinvaders_ppo.png")
                    torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")
                    
                    if np.mean(evaluation_reward) > max_eval:
                        torch.save(agent.policy_net.state_dict(), "./save_model/spaceinvaders_ppo_best")
                        max_eval = float(np.mean(evaluation_reward))
                        reset_count = 0
                    elif e > 5000:
                        reset_count += 1
                        """
                        if (reset_count == reset_max):
                            print("Training went nowhere, starting again at best model")
                            agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                            agent.update_target_net()
                            reset_count = 0
                        """
                e += 1
                evaluation_reward.append(env.score)
                print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                  " evaluation reward:", np.mean(evaluation_reward))
                
                env.done = False
                env.score = 0
                env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                env.state = env.reset()
                env.life = number_lives
                get_init_state(env.history, env.state)
                
                
                
    agent.train_policy_net(frame, frame_next_vals)
    agent.update_target_net()


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [16, 4, 8, 8], but got input of size [4, 84, 84] instead

In [ ]:
torch.save(agent.policy_net, "./save_model/spaceinvaders_ppo")

In [ ]:
### Loop through all environments and run PPO on them

env_names = ['SpaceInvaders-v0', 'MsPacman-v0', 'Asterix-v0', 'Asteroids-v0', 'Atlantis-v0', 'Alien-v0', 'Amidar-v0', 'Assault-v0', 'BankHeist-v0']

for a in range(len(env_names)):
    name = env_names[a]
    print("\n\n\n ------- STARTING TRAINING FOR %s ------- \n\n\n" % (name))
    
    envs = []
    for i in range(num_envs):
        envs.append(GameEnv(name))
    #env.render()
    

    number_lives = envs[0].life
    state_size = envs[0].observation_space.shape
    if (name == 'SpaceInvaders-v0'):
        action_size = 4
    else:
        action_size = envs[0].action_space.n
    rewards, episodes = [], []

    vis_env_idx = 0
    vis_env = envs[vis_env_idx]
    e = 0
    frame = 0
    max_eval = -np.inf
    reset_count = 0


    agent = Agent(action_size)
    torch.save(agent.policy_net.state_dict(), "./save_model/" + name + "_best")
    evaluation_reward = deque(maxlen=evaluation_reward_length)
    frame = 0
    memory_size = 0
    reset_max = 10
    
    print("Determing min/max rewards of environment")
    [low, high] = score_range = get_score_range(name)
    print("Min: %d. Max: %d." % (low, high))

    while (frame < 10000000):
        step = 0
        assert(num_envs * env_mem_size == train_frame)
        frame_next_vals = []
        
        for j in range(env_mem_size):
            
            curr_states = np.stack([envs[i].history[HISTORY_SIZE-1,:,:] for i in range(num_envs)])
            next_states = []
            net_in = np.stack([envs[i].history[:HISTORY_SIZE,:,:] for i in range(num_envs)])
            step += num_envs
            frame += num_envs
            actions, values = agent.get_action(np.float32(net_in) / 255.)
            
            for i in range(num_envs):
                env = envs[i]
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                next_states.append(next_state)
                if (i == vis_env_idx):
                    vis_env._env.render()
            
            for i in range(num_envs):
                env = envs[i]
                """
                next_state, env.reward, env.done, env.info = env.step(actions[i])
                if (i == vis_env_idx):
                    vis_env._env.render()
                """
                
                frame_next_state = get_frame(next_states[i])
                env.history[HISTORY_SIZE,:,:] = frame_next_state
                terminal_state = check_live(env.life, env.info['ale.lives'])
                env.life = env.info['ale.lives']
                r = env.reward #np.log(max(env.reward+1, 1))#((env.reward - low) / (high - low)) * 30
                agent.memory.push(i, deepcopy(curr_states[i]), actions[i], r, terminal_state, values[i], 0, 0)
                
                if (j == env_mem_size-1):
                    net_in = np.stack([envs[k].history[1:,:,:] for k in range(num_envs)])
                    _, frame_next_vals = agent.get_action(np.float32(net_in) / 255.)
                
                env.score += env.reward
                env.history[:HISTORY_SIZE, :, :] = env.history[1:,:,:]
        
                if (env.done):
                    if (e % 50 == 0):
                        print('now time : ', datetime.now())
                        rewards.append(np.mean(evaluation_reward))
                        episodes.append(e)
                        pylab.plot(episodes, rewards, 'b')
                        pylab.savefig("./save_graph/" + name + "_ppo.png")
                        torch.save(agent.policy_net, "./save_model/" + name + "_ppo")

                        if np.mean(evaluation_reward) > max_eval:
                            torch.save(agent.policy_net.state_dict(), "./save_model/"  + name + "_ppo_best")
                            max_eval = float(np.mean(evaluation_reward))
                            reset_count = 0
                        elif e > 5000:
                            reset_count += 1
                            """
                            if (reset_count == reset_max):
                                print("Training went nowhere, starting again at best model")
                                agent.policy_net.load_state_dict(torch.load("./save_model/spaceinvaders_ppo_best"))
                                agent.update_target_net()
                                reset_count = 0
                            """
                    e += 1
                    evaluation_reward.append(env.score)
                    print("episode:", e, "  score:", env.score,  " epsilon:", agent.epsilon, "   steps:", step,
                      " evaluation reward:", np.mean(evaluation_reward))

                    env.done = False
                    env.score = 0
                    env.history = np.zeros([HISTORY_SIZE+1,84,84], dtype=np.uint8)
                    env.state = env.reset()
                    env.life = number_lives
                    get_init_state(env.history, env.state)
            
        agent.train_policy_net(frame, frame_next_vals)
        agent.update_target_net()
    print("FINISHED TRAINING FOR %s" % (name))
    pylab.figure()




 ------- STARTING TRAINING FOR SpaceInvaders-v0 ------- 





/home/sigai/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Determing min/max rewards of environment
Min: 0. Max: 200.


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],1)[0].unsqueeze(1))


Training network. lr: 0.000250. clip: 0.100000


...

Iteration 6595: Policy loss: -0.002923. Value loss: 0.107544. Entropy: 0.883350.
Iteration 6596: Policy loss: -0.008312. Value loss: 0.070418. Entropy: 0.880668.
Iteration 6597: Policy loss: -0.012928. Value loss: 0.049538. Entropy: 0.878861.
episode: 3139   score: 210.0  epsilon: 1.0    steps: 592  evaluation reward: 240.7
episode: 3140   score: 250.0  epsilon: 1.0    steps: 648  evaluation reward: 241.4
Training network. lr: 0.000200. clip: 0.079881
Iteration 6598: Policy loss: 0.002715. Value loss: 0.132721. Entropy: 1.057269.
Iteration 6599: Policy loss: -0.001777. Value loss: 0.067534. Entropy: 1.053133.
Iteration 6600: Policy loss: -0.002685. Value loss: 0.067158. Entropy: 1.067686.
episode: 3141   score: 210.0  epsilon: 1.0    steps: 416  evaluation reward: 242.0
Training network. lr: 0.000199. clip: 0.079734
Iteration 6601: Policy loss: 0.002288. Value loss: 0.092244. Entropy: 0.859925.
Iteration 6602: Policy loss: -0.003227. Value loss: 0.062916. Entropy: 0.881289.
Iteration 6

episode: 3168   score: 135.0  epsilon: 1.0    steps: 576  evaluation reward: 238.05
episode: 3169   score: 315.0  epsilon: 1.0    steps: 608  evaluation reward: 239.1
Training network. lr: 0.000199. clip: 0.079577
Iteration 6655: Policy loss: 0.000777. Value loss: 0.173507. Entropy: 1.079863.
Iteration 6656: Policy loss: -0.004971. Value loss: 0.110098. Entropy: 1.088675.
Iteration 6657: Policy loss: -0.008208. Value loss: 0.084512. Entropy: 1.097745.
episode: 3170   score: 210.0  epsilon: 1.0    steps: 288  evaluation reward: 239.4
episode: 3171   score: 325.0  epsilon: 1.0    steps: 400  evaluation reward: 236.45
Training network. lr: 0.000199. clip: 0.079577
Iteration 6658: Policy loss: 0.000550. Value loss: 0.322772. Entropy: 0.976481.
Iteration 6659: Policy loss: -0.002113. Value loss: 0.194896. Entropy: 0.959239.
Iteration 6660: Policy loss: -0.006518. Value loss: 0.161574. Entropy: 0.962025.
episode: 3172   score: 275.0  epsilon: 1.0    steps: 72  evaluation reward: 237.05
Train

Iteration 6712: Policy loss: -0.000986. Value loss: 0.076793. Entropy: 0.930854.
Iteration 6713: Policy loss: -0.008322. Value loss: 0.046272. Entropy: 0.937302.
Iteration 6714: Policy loss: -0.009178. Value loss: 0.039014. Entropy: 0.913733.
episode: 3200   score: 240.0  epsilon: 1.0    steps: 144  evaluation reward: 230.3
now time :  2019-02-28 12:15:38.629922
episode: 3201   score: 215.0  epsilon: 1.0    steps: 568  evaluation reward: 229.5
Training network. lr: 0.000199. clip: 0.079421
Iteration 6715: Policy loss: 0.001381. Value loss: 0.129346. Entropy: 1.102507.
Iteration 6716: Policy loss: -0.003836. Value loss: 0.077815. Entropy: 1.100967.
Iteration 6717: Policy loss: -0.007944. Value loss: 0.062357. Entropy: 1.112944.
episode: 3202   score: 210.0  epsilon: 1.0    steps: 464  evaluation reward: 229.8
Training network. lr: 0.000199. clip: 0.079421
Iteration 6718: Policy loss: -0.001438. Value loss: 0.152248. Entropy: 0.873818.
Iteration 6719: Policy loss: -0.010012. Value loss: 

Iteration 6770: Policy loss: 0.003239. Value loss: 0.240761. Entropy: 1.190555.
Iteration 6771: Policy loss: -0.001017. Value loss: 0.196664. Entropy: 1.195369.
episode: 3231   score: 410.0  epsilon: 1.0    steps: 712  evaluation reward: 229.25
Training network. lr: 0.000198. clip: 0.079273
Iteration 6772: Policy loss: 0.000874. Value loss: 0.150582. Entropy: 0.812562.
Iteration 6773: Policy loss: -0.005375. Value loss: 0.078401. Entropy: 0.804930.
Iteration 6774: Policy loss: -0.005683. Value loss: 0.059888. Entropy: 0.805376.
episode: 3232   score: 210.0  epsilon: 1.0    steps: 176  evaluation reward: 229.25
Training network. lr: 0.000198. clip: 0.079273
Iteration 6775: Policy loss: -0.001253. Value loss: 0.162700. Entropy: 1.088054.
Iteration 6776: Policy loss: -0.006257. Value loss: 0.099457. Entropy: 1.072156.
Iteration 6777: Policy loss: -0.011905. Value loss: 0.102922. Entropy: 1.088286.
episode: 3233   score: 180.0  epsilon: 1.0    steps: 248  evaluation reward: 229.7
episode: 

Iteration 6830: Policy loss: -0.004599. Value loss: 0.076520. Entropy: 1.164457.
Iteration 6831: Policy loss: -0.009284. Value loss: 0.065921. Entropy: 1.150126.
episode: 3260   score: 235.0  epsilon: 1.0    steps: 952  evaluation reward: 231.1
Training network. lr: 0.000198. clip: 0.079117
Iteration 6832: Policy loss: -0.001764. Value loss: 0.103252. Entropy: 0.838621.
Iteration 6833: Policy loss: -0.005017. Value loss: 0.075917. Entropy: 0.841432.
Iteration 6834: Policy loss: -0.009494. Value loss: 0.057222. Entropy: 0.853134.
episode: 3261   score: 210.0  epsilon: 1.0    steps: 80  evaluation reward: 232.0
episode: 3262   score: 240.0  epsilon: 1.0    steps: 864  evaluation reward: 232.3
Training network. lr: 0.000198. clip: 0.079117
Iteration 6835: Policy loss: -0.000067. Value loss: 0.147189. Entropy: 1.047356.
Iteration 6836: Policy loss: -0.003323. Value loss: 0.092009. Entropy: 1.060730.
Iteration 6837: Policy loss: -0.005114. Value loss: 0.098826. Entropy: 1.064236.
episode: 3